In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")  
for gpu in gpus:
    # 设置内存增长方式 自增长
    tf.config.experimental.set_memory_growth(gpu, True)  

In [3]:
file_path = '../datasets/红楼梦/红楼梦.txt'
text = open(file_path, 'r', encoding='utf-8').read()

print('text length:', len(text))
print(text[:200])

text length: 855324
第一回  甄士隐梦幻识通灵　贾雨村风尘怀闺秀


    此开卷第一回也．作者自云：因曾历过一番梦幻之后，故将真事隐去，而借"通灵"之说，撰此《石头记》一书也．故曰"甄士隐"云云．但书中所记何事何人？自又云：“今风尘碌碌，一事无成，忽念及当日所有之女子，一一细考较去，觉其行止见识，皆出于我之上．何我堂堂须眉，诚不若彼裙钗哉？实愧则有余，悔又无益之大无可如何之日也！当此，则自欲将已往所赖天恩祖德，锦


In [4]:
#import jieba
#text = jieba.lcut(text)
#print( text[:50] )
#import nltk
#nltk.download('punkt')
#tokens = nltk.word_tokenize(text)
#print( tokens[:50])

In [5]:
# 生成词表 char -> id
vocab = sorted( set(text) )
print(len(vocab))
print(vocab[:300])

4532
['\t', '\n', ' ', '!', '"', '(', ')', '*', ',', '-', '.', '5', '7', ':', ';', '>', '?', 'V', 'W', '_', '`', 'f', 'l', 'r', 'z', '|', '~', 'ǎ', 'Τ', 'σ', 'τ', 'Ё', 'Б', 'Г', 'Д', 'Е', 'Ж', 'З', 'К', 'Л', 'М', 'Н', 'О', 'Р', 'С', 'Т', 'У', 'Ф', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '—', '‘', '’', '“', '”', '…', '∪', '⑴', '⑷', '⑹', '⒍', '⒚', '\u3000', '、', '。', '〉', '《', '》', 'ぁ', 'あ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'の', 'べ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'コ', 'ゴ', 'サ', 'シ', 'ス', 'ズ', 'ゼ', 'ゾ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'ニ', 'ヌ', 'ネ', 'ノ', 'パ', 'ピ', 'ブ', 'プ', 'ペ', 'ㄉ', 'ㄖ', '一', '丁', '七

In [6]:
filter_char = [ 'ǎ', 'Τ', 'σ', 'τ', 'Ё', 'Б', 'Г', 'Д', 'Е', 'Ж', 'З', 'К', 'Л', 'М', 'Н', 'О', 'Р', 'С', 'Т', 'У', 'Ф', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '—', '‘', '’', '“', '”', '…', '∪', '⑴', '⑷', '⑹', '⒍', '⒚', '\u3000', '、', '。', '〉', '《', '》', 'ぁ', 'あ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'の', 'べ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'コ', 'ゴ', 'サ', 'シ', 'ス', 'ズ', 'ゼ', 'ゾ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'ニ', 'ヌ', 'ネ', 'ノ', 'パ', 'ピ', 'ブ', 'プ', 'ペ', 'ㄉ', 'ㄖ']
text = list( filter(lambda a: a not in filter_char, text) )
print( text[:300] )

['第', '一', '回', ' ', ' ', '甄', '士', '隐', '梦', '幻', '识', '通', '灵', '贾', '雨', '村', '风', '尘', '怀', '闺', '秀', '\n', '\n', '\n', ' ', ' ', ' ', ' ', '此', '开', '卷', '第', '一', '回', '也', '．', '作', '者', '自', '云', '：', '因', '曾', '历', '过', '一', '番', '梦', '幻', '之', '后', '，', '故', '将', '真', '事', '隐', '去', '，', '而', '借', '"', '通', '灵', '"', '之', '说', '，', '撰', '此', '石', '头', '记', '一', '书', '也', '．', '故', '曰', '"', '甄', '士', '隐', '"', '云', '云', '．', '但', '书', '中', '所', '记', '何', '事', '何', '人', '？', '自', '又', '云', '：', '今', '风', '尘', '碌', '碌', '，', '一', '事', '无', '成', '，', '忽', '念', '及', '当', '日', '所', '有', '之', '女', '子', '，', '一', '一', '细', '考', '较', '去', '，', '觉', '其', '行', '止', '见', '识', '，', '皆', '出', '于', '我', '之', '上', '．', '何', '我', '堂', '堂', '须', '眉', '，', '诚', '不', '若', '彼', '裙', '钗', '哉', '？', '实', '愧', '则', '有', '余', '，', '悔', '又', '无', '益', '之', '大', '无', '可', '如', '何', '之', '日', '也', '！', '当', '此', '，', '则', '自', '欲', '将', '已', '往', '所', '赖', '天', '恩', '祖', '德', '，', '锦', '衣', '纨', '绔', '

In [7]:
# 生成词表 char -> id
vocab = sorted( set(text) )
print(len(vocab))
print(vocab[:300])

4364
['\t', '\n', ' ', '!', '"', '(', ')', '*', ',', '-', '.', '5', '7', ':', ';', '>', '?', 'V', 'W', '_', '`', 'f', 'l', 'r', 'z', '|', '~', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '丐', '丑', '专', '且', '世', '丘', '丙', '业', '丛', '东', '丝', '丞', '丢', '两', '严', '丧', '个', '丫', '中', '丰', '串', '临', '丸', '丹', '为', '主', '丽', '举', '乃', '久', '乇', '么', '义', '之', '乌', '乍', '乎', '乏', '乐', '乔', '乖', '乘', '乙', '乜', '九', '乞', '也', '习', '乡', '书', '乩', '买', '乱', '乳', '乾', '了', '予', '争', '事', '二', '于', '亏', '云', '互', '五', '井', '亘', '些', '亡', '亢', '交', '亥', '亦', '产', '亩', '享', '京', '亭', '亮', '亲', '亵', '人', '亿', '什', '仁', '仃', '仄', '仅', '仆', '仇', '今', '介', '仍', '从', '仓', '仔', '仕', '他', '仗', '付', '仙', '代', '令', '以', '仪', '们', '仰', '仲', '仵', '件', '价', '任', '份', '仿', '伊', '伍', '伏', '伐', '休', '众', '优', '伙', '会', '伞', '伟', '传', '伤', '伦', '伫', '伯', '估', '伴', '伶', '伸', '伺', '似', '伽', '但', '位', '低', '住', '佐', '佑', '体', '何', '余', '佚', '佛', '作', '佞', '你', '佣', '佩', '佯', '佳', '佻', '使', '侄', '侈', '例', '侍', '侑

In [8]:
char_to_id = {char:id for id, char in enumerate(vocab)}
print(char_to_id)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '5': 11, '7': 12, ':': 13, ';': 14, '>': 15, '?': 16, 'V': 17, 'W': 18, '_': 19, '`': 20, 'f': 21, 'l': 22, 'r': 23, 'z': 24, '|': 25, '~': 26, '一': 27, '丁': 28, '七': 29, '万': 30, '丈': 31, '三': 32, '上': 33, '下': 34, '不': 35, '与': 36, '丐': 37, '丑': 38, '专': 39, '且': 40, '世': 41, '丘': 42, '丙': 43, '业': 44, '丛': 45, '东': 46, '丝': 47, '丞': 48, '丢': 49, '两': 50, '严': 51, '丧': 52, '个': 53, '丫': 54, '中': 55, '丰': 56, '串': 57, '临': 58, '丸': 59, '丹': 60, '为': 61, '主': 62, '丽': 63, '举': 64, '乃': 65, '久': 66, '乇': 67, '么': 68, '义': 69, '之': 70, '乌': 71, '乍': 72, '乎': 73, '乏': 74, '乐': 75, '乔': 76, '乖': 77, '乘': 78, '乙': 79, '乜': 80, '九': 81, '乞': 82, '也': 83, '习': 84, '乡': 85, '书': 86, '乩': 87, '买': 88, '乱': 89, '乳': 90, '乾': 91, '了': 92, '予': 93, '争': 94, '事': 95, '二': 96, '于': 97, '亏': 98, '云': 99, '互': 100, '五': 101, '井': 102, '亘': 103, '些': 104, '亡': 105, '亢': 106, '交': 107, '亥': 108, '亦': 109, '产': 11

In [9]:
ids_of_text = np.array( [char_to_id[char] for char in text] )
print(text[:20])
print(ids_of_text[:20])

['第', '一', '回', ' ', ' ', '甄', '士', '隐', '梦', '幻', '识', '通', '灵', '贾', '雨', '村', '风', '尘', '怀', '闺']
[2748   27  724    2    2 2439  812 4061 1886 1156 3507 3816 2227 3630
 4081 1791 4166 1036 1269 4006]


In [10]:
def split_input_target(text_ids):
    # [ 1 2 3 4 5] -> [1 2 3 4], [2 3 4 5]
    return text_ids[:-1], text_ids[1:]
seq_length = 100
seq_num = len(ids_of_text) // seq_length
ids_of_text = ids_of_text[:seq_length * seq_num]
print( ids_of_text.shape )

(827200,)


In [11]:
ids_of_text = ids_of_text.reshape(-1, seq_length)
print(ids_of_text.shape)
print(ids_of_text[0])

(8272, 100)
[2748   27  724    2    2 2439  812 4061 1886 1156 3507 3816 2227 3630
 4081 1791 4166 1036 1269 4006 2664    1    1    1    2    2    2    2
 1977 1190  474 2748   27  724   83 4357  185 2984 3096   99 4359  726
 1757  480 3776   27 2467 1886 1156   70  542 4356 1645 1029 2546   95
 4061  494 4356 2986  240    4 3816 2227    4   70 3544 4356 1623 1977
 2587  838 3493   27   86   83 4357 1645 1750    4 2439  812 4061    4
   99   99 4357  174   86   55 1417 3493  181   95  181  118 4361 3096
  497   99]


In [12]:
seq_ids = map(split_input_target, ids_of_text)
seq_ids = np.array(list(seq_ids))
print(seq_ids.shape)
print(seq_ids[0])

(8272, 2, 99)
[[2748   27  724    2    2 2439  812 4061 1886 1156 3507 3816 2227 3630
  4081 1791 4166 1036 1269 4006 2664    1    1    1    2    2    2    2
  1977 1190  474 2748   27  724   83 4357  185 2984 3096   99 4359  726
  1757  480 3776   27 2467 1886 1156   70  542 4356 1645 1029 2546   95
  4061  494 4356 2986  240    4 3816 2227    4   70 3544 4356 1623 1977
  2587  838 3493   27   86   83 4357 1645 1750    4 2439  812 4061    4
    99   99 4357  174   86   55 1417 3493  181   95  181  118 4361 3096
   497]
 [  27  724    2    2 2439  812 4061 1886 1156 3507 3816 2227 3630 4081
  1791 4166 1036 1269 4006 2664    1    1    1    2    2    2    2 1977
  1190  474 2748   27  724   83 4357  185 2984 3096   99 4359  726 1757
   480 3776   27 2467 1886 1156   70  542 4356 1645 1029 2546   95 4061
   494 4356 2986  240    4 3816 2227    4   70 3544 4356 1623 1977 2587
   838 3493   27   86   83 4357 1645 1750    4 2439  812 4061    4   99
    99 4357  174   86   55 1417 3493  181 

In [13]:
train_data = np.array( [item[0] for item in seq_ids] )
train_label = np.array( [item[1] for item in seq_ids])

In [14]:
# Create rnn model
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024
batch_size = 64
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim,
                           batch_input_shape = [batch_size, None]),
    keras.layers.GRU(units = rnn_units, stateful = True, return_sequences = True),
    keras.layers.Dense(vocab_size, activation = 'softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           558592    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3545088   
_________________________________________________________________
dense (Dense)                (64, None, 4364)          4473100   
Total params: 8,576,780
Trainable params: 8,576,780
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy']
)

In [16]:
step = train_data.shape[0] // batch_size
train_data = train_data[:step * batch_size]
train_label = train_label[:step * batch_size]

In [17]:
import os
callbacks_dir = './callbacks'
if not os.path.exists(callbacks_dir):
    os.makedirs(callbacks_dir)
best_model_file_path = os.path.join(callbacks_dir, 'best_text_generate_stone_model.h5')
# load saved model

try:
    model.load_weights(best_model_file_path)
    print('Load weights suc! Continue to fit model.')
except:
    print('Load weights failed! Start to fit new model.')

callbacks = [
    keras.callbacks.EarlyStopping(min_delta=1e-3, patience=10, monitor='loss'),
    keras.callbacks.ModelCheckpoint(best_model_file_path, save_best_only=True, monitor='loss')  
] 

Load weights suc! Continue to fit model.


In [18]:
history = model.fit(train_data, train_label,
                    epochs = 10,
                    batch_size = batch_size,
                    callbacks = callbacks
)

Train on 8256 samples
Epoch 1/10
8256/8256 [==============================] - 23s 3ms/sample - loss: 1.2938 - accuracy: 0.7173
Epoch 2/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.2227 - accuracy: 0.7380
Epoch 3/10
8256/8256 [==============================] - 19s 2ms/sample - loss: 1.2032 - accuracy: 0.7429
Epoch 4/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.1882 - accuracy: 0.7458
Epoch 5/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.1756 - accuracy: 0.7480
Epoch 6/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.1657 - accuracy: 0.7495
Epoch 7/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.1602 - accuracy: 0.7493
Epoch 8/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.1508 - accuracy: 0.7508
Epoch 9/10
8256/8256 [==============================] - 18s 2ms/sample - loss: 1.1408 - accuracy: 0.7535
Epoch 10/10
8256/8256 [==========

In [19]:
predictions = model.predict_classes(train_data[0:batch_size], batch_size)

In [20]:
print(train_label[0])
print(predictions[0])

[  27  724    2    2 2439  812 4061 1886 1156 3507 3816 2227 3630 4081
 1791 4166 1036 1269 4006 2664    1    1    1    2    2    2    2 1977
 1190  474 2748   27  724   83 4357  185 2984 3096   99 4359  726 1757
  480 3776   27 2467 1886 1156   70  542 4356 1645 1029 2546   95 4061
  494 4356 2986  240    4 3816 2227    4   70 3544 4356 1623 1977 2587
  838 3493   27   86   83 4357 1645 1750    4 2439  812 4061    4   99
   99 4357  174   86   55 1417 3493  181   95  181  118 4361 3096  497
   99]
[  83 1692 4356    2 1847  984 3801 1400 1156 3507 1706 1399 3630   97
 1791   97 1036 1269 4006 2664    1    1    2    2    2    2    2    0
  118  474 2748 3671  724   83 2048 4330 2984 3096 2701 4359   20 1757
  480 3776   27 2467 4356 1156   70  542 4356 1645 1029 3096 3526 4061
  494 4356 2986  494 1977 3816 2227    4   70 3544 4356 1623 1977 2587
  838 3493 4356   86   83 4357 1645 1750    4 2439  812 4061    4   99
   99 4357    1 3493   55  181 3493  181  832   83 3096 4361 1645  545

In [21]:
single_batch_size = 1
single_batch_model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim,
                           batch_input_shape = [single_batch_size, None]),
    keras.layers.GRU(units = rnn_units, stateful = True, return_sequences = True),
    keras.layers.Dense(vocab_size, activation = 'softmax')
])
single_batch_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            558592    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3545088   
_________________________________________________________________
dense_1 (Dense)              (1, None, 4364)           4473100   
Total params: 8,576,780
Trainable params: 8,576,780
Non-trainable params: 0
_________________________________________________________________


In [22]:
single_batch_model.load_weights(best_model_file_path)
single_batch_model.build(input_shape=(1,))

In [23]:
pred = single_batch_model.predict_classes(train_data[0:1], batch_size=1)

In [24]:
print(pred)
print(train_label[0])

[[ 233 1143 3450    2    2  833  210 1886  549 3507 3816  127 1286 2002
  1791 1135 1036 1269 4006 2664    1    1    2    2    2    2    2    0
  1699 4208 2748   27  724   83 2048  999 2984 3096   99 4359  726 1032
   480 3776 4356 2467 4356 1156   70  542 4356 1645 1029 3096 3526 4061
  2739 4356 2986   40 1977 3816 2227    4   70 1645 4356 1623   27 2587
   838 3493 4356   86   83 4357 1645 1750 4359 2439  812 4061    4   99
    99 4359  984   86   55 1417 3493  181  181  181 1645 4361 1645  130
    99]]
[  27  724    2    2 2439  812 4061 1886 1156 3507 3816 2227 3630 4081
 1791 4166 1036 1269 4006 2664    1    1    1    2    2    2    2 1977
 1190  474 2748   27  724   83 4357  185 2984 3096   99 4359  726 1757
  480 3776   27 2467 1886 1156   70  542 4356 1645 1029 2546   95 4061
  494 4356 2986  240    4 3816 2227    4   70 3544 4356 1623 1977 2587
  838 3493   27   86   83 4357 1645 1750    4 2439  812 4061    4   99
   99 4357  174   86   55 1417 3493  181   95  181  118 4361 

In [25]:
print(''.join(vocab[id] for id in train_label[0]))
print('----------')
print(''.join(vocab[id] for id in pred[0]))

一回  甄士隐梦幻识通灵贾雨村风尘怀闺秀


    此开卷第一回也．作者自云：因曾历过一番梦幻之后，故将真事隐去，而借"通灵"之说，撰此石头记一书也．故曰"甄士隐"云云．但书中所记何事何人？自又云
----------
倍幅西  夫便梦君识通今性母村带尘怀闺秀

     	时香第一回也没家者自云：因小历过，番，幻之后，故将自话隐笑，而且此通灵"之故，撰一石头记，书也．故曰：甄士隐"云云：宝书中所记何何何故？故从云


In [26]:
# 文本生成，每次生成一个字符，并对结果进行抽样（否则结果是固定的，容易陷入循环）
def generate_text(model, start_string, num_generate = 500):
    input_data = [char_to_id[char] for char in start_string]
    input_data = tf.expand_dims(input_data, 0)
    
    text_generated = []
    model.reset_states()
    
    for _ in range(num_generate):
        # 模型前向传播 -> 推测出下一个字符（可能性）
        # -> 抽样 -> 生成下一个字符 -> 更新输入数据，准备下一次前向传播
        predictions = model(input_data)
        predictions = tf.squeeze(predictions, 0)
        prediction = predictions[-1]
        
        # 抽样
        prediction = prediction.numpy() #[0.01, 0.94...]
        predicted_id = np.random.choice(range(len(prediction)), 1, p = prediction)[0]
        
        #生成下个字符
        text_generated.append( vocab[predicted_id] )
        
        #更新输入数据，准备下一次前向传播
        input_data = tf.expand_dims([predicted_id], 0)
    
    return start_string + ''.join(text_generated)

new_text = generate_text(single_batch_model, "贾宝玉")
print(new_text)

贾宝玉捱了赵姨娘嘻的推他走了．他在袭人身后轻轻，儿刚平儿出光！凤姐与秋纹拉了丫头的信儿进去呢．宝玉便站住两眼红线，问宝玉不会，恐金钏儿满屋紫鹃也将那收着的时候，也不过去．这会子你们别人也没用开，不管乱叫个菱，才叫他！我的新衣裳去，只怕遇着太太的了薛蝌回复了已到，将来商议已办，不容分分娘，不免又回兄弟致放香京舍省城．正没大夫，见了凤姐贤良，因他便破了小子，在当面各行旧年，又都偷空收拾，不知不是那里弄出来的，珍大爷亲自看见一个磕头子，再不许说人知道，就没那计为没人还强见过来！正是二十来了这里，你又带了去进来说：环兄弟，我还刚才打嘴．如今看见老爷家得你宝玉老爷去，又恐叫人来照应．"邢夫人便回说：大太太作兴儿．可惜你们三个各自归房去的，就惹出他些，也有想着自己安歇，好好的，如今竟得省些找他一碗气宝玉也告诉了他，便说：你也要许多心．这些事虽有人心瘦，多彼糊涂，人说这个话也不说．且素日你们有这个管事正说着，都来和黛玉面西，不放手，便上来闭个眼诉了人，又说道：你来了，我略带了人去打合他婆子见凤姐儿来，心里没气发，哭道：你破嘴说的是主意的，不厌你老人家就好，亏你怎么说？我倒不怪臊，这不作什么黛玉忙道：又急了
